
# Taller (3h) — FastAPI + Scikit-Learn orientado a investigación
**Objetivo:** Diseñar, entrenar y servir un modelo de ML **investigando** las decisiones técnicas clave.  
**Entrega:** API funcional (FastAPI) + breve informe (markdown dentro del notebook) justificando decisiones.

> Filosofía del taller: menos receta, más criterio. No hay una única respuesta correcta; lo evaluado es la **calidad del razonamiento**, la **limpieza de la implementación** y la **capacidad de probar** la API.



## Agenda sugerida (3h)
1) **Planteo del problema y dataset** (30–40 min)  
2) **Entrenamiento y persistencia** (40–50 min)  
3) **Diseño del contrato y API** (45–55 min)  
4) **Pruebas, errores y mejoras** (30–35 min)

### Reglas
- No borres los encabezados `TODO`. Agrega tu código debajo de cada bloque indicado.
- Documenta tus decisiones en la sección **Bitácora** al final.
- Puedes trabajar en equipo, pero cada entrega debe ser individual y original.



---
## 1) Selección de dataset y formulación del problema (investigación)
Elige **uno**:
- A) `sklearn.datasets.load_wine` (clasificación 3 clases, baseline rápido)
- B) Un dataset tabular de UCI, Kaggle u otra fuente **citable** (debe ser pequeño/mediano y con licencia apta)
- C) Un CSV propio (explica origen y variables)

**Requisitos mínimos:**
- Problema de **clasificación** o **regresión** tabular
- Al menos **6 features numéricas** (puedes convertir categóricas)
- Justifica por qué es un buen caso para servir vía API

**Entrega (en esta celda, texto breve):** Describe el dataset, objetivo, variables y métrica principal.



### 1.1 Carga y exploración (TODO)
- Carga el dataset (pd.read_csv o loader de sklearn).
- Muestra `head()`, `describe()` y verifica nulos/outliers.
- Selecciona `X` (features) y `y` (target); explica tu elección.


In [ ]:
import pandas as pd

class TrainModel():
    def load_data(self, path="data/Churn.xlsx"):
        """Cargar dataset"""
        df = pd.read_excel(path)
        return df

    def explore_data(self, df):
        """Explorar dataset."""
        print("Primeras filas:\n", df.head())
        print("\nInformación del dataset:\n", df.describe())
        print("\nDistribución de la variable objetivo 'Churn':\n", df['Churn'].value_counts(normalize=True))

    def preprocess_data(self, df):
        # Transformación
        df['Intl_Plan'] = df['Intl_Plan'].map({'yes': 1, 'no': 0})
        df['Vmail_Plan'] = df['Vmail_Plan'].map({'yes': 1, 'no': 0})
        df['Churn'] = df['Churn'].map({'True.': 1, 'False.': 0})

        # Nuevas variables derivadas
        df['Total_Calls'] = df['Day_Calls'] + df['Eve_Calls'] + df['Night_Calls'] + df['Intl_Calls']
        df['Total_Mins'] = df['Day_Mins'] + df['Eve_Mins'] + df['Night_Mins'] + df['Intl_Mins']
        df['Total_Charge'] = df['Day_Charge'] + df['Eve_Charge'] + df['Night_Charge'] + df['Intl_Charge']
        df['High_Usage'] = (df['Total_Charge'] > df['Total_Charge'].mean()).astype(int)
        df['Many_CustServ_Calls'] = (df['CustServ_Calls'] > 5).astype(int)

        df = df.drop(columns=['Phone', 'State','Day_Calls', 'Eve_Calls', 'Night_Calls','Intl_Calls',   
                                'Day_Mins', 'Eve_Mins', 'Night_Mins','Intl_Mins', 
                                'Day_Charge', 'Eve_Charge','Night_Charge', 'Intl_Charge'])
        # Eliminar nulos
        df = df.dropna()

        X = df.drop('Churn', axis=1)
        y = df['Churn']

        return X, y



---
## 2) Entrenamiento y persistencia del modelo (investigación)
Toma decisiones y **justifícalas**:
- ¿Modelo base? (p. ej. `LogisticRegression`, `RandomForest`, `XGBoost` si lo instalas)
- ¿Preprocesamiento? (escala, imputación, OneHot, etc.)
- ¿Validación? (`train_test_split` vs `cross_val_score`)
- ¿Métrica? (clasificación: accuracy/F1; regresión: RMSE/MAE…)

**Requisito:** empaqueta tu flujo en un `Pipeline` de sklearn y **persiste** el modelo y columnas (joblib + JSON).




In [ ]:
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_validate
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import joblib

MODEL_PATH = "models/trainModel.pkl"


class TrainModel():
    def split_data(self, X, y, test_size=0.3, random_state=42):
        """Dividir dataset en train/test."""
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state, stratify=y)
        return X_train, X_test, y_train, y_test

    def create_model_pipeline(self, max_depth=5, min_samples_leaf=10, min_samples_split=20):
        """Crear pipeline árbol de decisión."""
        pipeline = Pipeline(steps=[
            ('smote', SMOTE(random_state=42)),
            ('clf', DecisionTreeClassifier(max_depth=max_depth, min_samples_leaf=min_samples_leaf,min_samples_split=min_samples_split, random_state=42))
        ])
        return pipeline

    def train_model(self, pipeline, X_train, y_train):
        """Entrenar modelo de árbol de decisión."""
        pipeline.fit(X_train, y_train)
        return pipeline

    def evaluate_model(self, model, X_test, y_test):
        """Evaluar modelo en test y mostrar métricas."""
        y_pred = model.predict(X_test)
        print("\n📊 Reporte de Clasificación en Test:\n")
        print(classification_report(y_test, y_pred))
        return y_pred

    def cross_validate_model(self, model, X_train, y_train):
        """Cross-validation 5 folds en train."""
        cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
        scoring = ['accuracy', 'precision', 'recall', 'f1']
        cv_results = cross_validate(model, X_train, y_train, cv=cv, scoring=scoring)
        print("\n📊 Resultados de cross-validation (promedio en 5 folds):")
        for metric in scoring:
            print(f"{metric}: {cv_results['test_' + metric].mean():.3f}")

    def create_model_visualization(self, model, X, y_test, y_pred):
        """Visualización del árbol y matriz de confusión."""
        plt.figure(figsize=(20, 8))
        plot_tree(
            model.named_steps['clf'],
            feature_names=X.columns,
            class_names=["Churn", "No Churn"],
            filled=True,
            rounded=True,
            fontsize=10
        )
        plt.title("Árbol de Decisión - Churn")
        plt.show()

        # Matriz de confusión
        cm = confusion_matrix(y_test, y_pred)
        disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=["Churn", "No Churn"])
        disp.plot(cmap='Blues')
        plt.title("Matriz de Confusión - Árbol de Decisión")
        plt.show()

    def save_model(self, model, columns, path=MODEL_PATH):
        """Guardar modelo + columnas usadas."""
        joblib.dump({
            "model": model,
            "columns": columns
        }, path)
        print(f"\n✅ Modelo guardado en {path}")

    # ------------------------------
    # Pipeline principal
    # ------------------------------

    def main(self):
        df = self.load_data()
        self.explore_data(df)
        X, y = self.preprocess_data(df)
        X_train, X_test, y_train, y_test = self.split_data(X, y)
            
        pipeline = self.create_model_pipeline()
        model = self.train_model(pipeline, X_train, y_train)
            
        y_pred = self.evaluate_model(model, X_test, y_test)
        self.cross_validate_model(model, X_train, y_train)
            
        self.create_model_visualization(model, X, y_test, y_pred)
        self.save_model(model, X.columns.tolist())
    
    if __name__ == "__main__":
        trainer = TrainModel()
        trainer.main()




---
## 3) Diseño del contrato de la API (investigación)
Define **endpoints mínimos**:
- `GET /health` (estado, versión del modelo, métrica)
- `POST /predict` (un registro)
- `POST /predict-batch` (lista de registros)

**Decisiones a justificar:**
- ¿Qué validaciones aplicas en `Pydantic`? (rango, tipos, campos extra)
- ¿Cómo garantizas el **orden** de columnas?
- ¿Qué devuelves además de la predicción? (probabilidades, latencia, advertencias)


from pydantic import BaseModel, Field

class ChurnFeatures(BaseModel):
    Account_Length: int = Field(..., ge=0, le=300)
    Area_Code: int = Field(..., ge=100, le=999)
    Intl_Plan: int = Field(..., ge=0, le=1)
    Vmail_Plan: int = Field(..., ge=0, le=1)
    Vmail_Message: int = Field(..., ge=0, le=200)
    CustServ_Calls: int = Field(..., ge=0, le=20)
    Total_Calls: float = Field(..., ge=0, le=1000)
    Total_Mins: float = Field(..., ge=0, le=2000)
    Total_Charge: float = Field(..., ge=0, le=500)
    High_Usage: int = Field(..., ge=0, le=1)
    Many_CustServ_Calls: int = Field(..., ge=0, le=1)


class ChurnResponse(BaseModel):
    churn_prediction: int
    churn_probability: float



In [ ]:
# Schema
from pydantic import BaseModel, Field

class ChurnFeatures(BaseModel):
    Account_Length: int = Field(..., ge=0, le=300)
    Area_Code: int = Field(..., ge=100, le=999)
    Intl_Plan: int = Field(..., ge=0, le=1)
    Vmail_Plan: int = Field(..., ge=0, le=1)
    Vmail_Message: int = Field(..., ge=0, le=200)
    CustServ_Calls: int = Field(..., ge=0, le=20)
    Total_Calls: float = Field(..., ge=0, le=1000)
    Total_Mins: float = Field(..., ge=0, le=2000)
    Total_Charge: float = Field(..., ge=0, le=500)
    High_Usage: int = Field(..., ge=0, le=1)
    Many_CustServ_Calls: int = Field(..., ge=0, le=1)

class ChurnResponse(BaseModel):
    churn_prediction: int
    churn_probability: float
    warnings : list




---
## 4) Implementación de FastAPI (investigación)
Crea un archivo `app.py` con:
- Carga perezosa de `model.joblib` y `feature_columns.json`
- Esquemas Pydantic (v2)
- Endpoints `/health`, `/predict`, `/predict-batch`
- Manejo de errores con `HTTPException` y mensajes claros

**Pistas** (no copiar/pegar sin entender):
- `model = joblib.load(...)`
- `class Sample(BaseModel): ...`
- `model.predict` y/o `model.predict_proba`
- Retornar JSON con `dict | BaseModel`

**Requisito:** esta celda **debe** escribir `app.py` con al menos la estructura básica.


In [ ]:
import time
import logging
import pandas as pd
import numpy as np
import joblib
from fastapi import FastAPI, Request, HTTPException
from fastapi.responses import JSONResponse,FileResponse
from src.schemas import ChurnFeatures, ChurnBatch
from src.utils.validateDataEndpoint import validate_churn_features


app = FastAPI(title="Churn Prediction API")

# Cargar modelo
artifacts = joblib.load("models/trainModel.pkl")
model = artifacts["model"]
columns = artifacts["columns"]

# Configurar logger básico
logging.basicConfig(
    format='%(asctime)s - %(levelname)s - %(message)s',
    level=logging.INFO
)

# -----------------------------
# Middleware de latencia y logger
# -----------------------------
@app.middleware("http")
async def add_process_time_header(request: Request, call_next):
    start_time = time.time()
    response = await call_next(request)
    process_time_ms = (time.time() - start_time) * 1000
    response.headers["X-Process-Time-ms"] = f"{process_time_ms:.2f}"
    
    # Log estructurado
    logging.info({
        "path": request.url.path,
        "method": request.method,
        "status_code": response.status_code,
        "process_time_ms": round(process_time_ms, 2)
    })
    return response

@app.get("/health")
def health():
    """Endpoint de estado de la API"""
    return {"status": "ok", "model_loaded": bool(model)}

# -----------------------------
# Endpoint de predicción
# -----------------------------
@app.post("/predict")
def predict(data: ChurnFeatures):
    try:

        validate_churn_features(data)

        input_data = np.array([[ 
            data.Account_Length,
            data.Area_Code,
            data.Intl_Plan,
            data.Vmail_Plan,
            data.Vmail_Message,
            data.CustServ_Calls,
            data.Total_Calls,
            data.Total_Mins,
            data.Total_Charge,
            data.High_Usage,
            data.Many_CustServ_Calls
        ]])

        prediction = model.predict(input_data)[0]
        probability = model.predict_proba(input_data)[:, 1][0]

        return JSONResponse({
            "churn_prediction": int(prediction),
            "churn_probability": float(probability)
        })

    except HTTPException as e:
        raise e

    except Exception as e:
        return JSONResponse(status_code=500, content={"detail": f"Error al predecir: {str(e)}"})


@app.post("/predict_batch")
def predict_batch(batch_data: ChurnBatch):
    """Predicción para múltiples registros"""
    try:
        results = []
        for record in batch_data.batch:
            
            validate_churn_features(record)

            input_data = np.array([[
                record.Account_Length,
                record.Area_Code,
                record.Intl_Plan,
                record.Vmail_Plan,
                record.Vmail_Message,
                record.CustServ_Calls,
                record.Total_Calls,
                record.Total_Mins,
                record.Total_Charge,
                record.High_Usage,
                record.Many_CustServ_Calls
            ]])
            prediction = model.predict(input_data)[0]
            probability = model.predict_proba(input_data)[:, 1][0]
            results.append({
                "prediction": int(prediction),
                "probability": float(probability)
            })
        return {"results": results}

    except HTTPException as e:
        raise e

    except Exception as e:
        return JSONResponse(status_code=500, content={"detail": f"Error al predecir: {str(e)}"})


@app.get("/export_predictions")
def export_predictions():

    """Genera y devuelve un Excel con predicciones usando las columnas guardadas en el joblib."""

    df = pd.DataFrame(columns=artifacts["columns"]) 
    df["predicted_churn"] = model.predict(df)
    df["probability_churn"] = model.predict_proba(df)[:, 1]

    output_path = "predicciones_churn.xlsx"
    df.to_excel(output_path, index=False)

    return FileResponse(
        output_path,
        filename="predicciones_churn.xlsx",
        media_type="application/vnd.openxmlformats-officedocument.spreadsheetml.sheet"
    )




### 4.1 Ejecutar el servidor
```bash
uvicorn app:app --reload --port 8000
```
Abre `http://127.0.0.1:8000/docs` y prueba manualmente. Registra resultados.



---
## 5) Pruebas y casos límite (investigación)
Define y ejecuta **al menos 6** pruebas:
- 3 válidas (predict y batch)
- 3 inválidas (campo faltante, tipo incorrecto, campo extra, etc.)

Incluye código de prueba y captura de respuestas.


In [ ]:

import requests
import json

BASE_URL = "http://127.0.0.1:8000"

# Cargar payloads desde archivo
with open("./fixtures/payloads.json", "r") as file:
    payloads = json.load(file)

# ----------------------------
# Health check
# ----------------------------
response = requests.get(f"{BASE_URL}/health")
print("\n=== Health Check ===")
print("Status Code:", response.status_code)
print("Response:", response.json())

# ----------------------------
# Predicción individual (tomamos el primer objeto del JSON)
# ----------------------------
first_payload = payloads[0]  # primer objeto del array de payloads

response = requests.post(f"{BASE_URL}/predict", json=first_payload)
print("\n=== Individual Prediction ===")
print("Payload:", json.dumps(first_payload, indent=4))
print("Status Code:", response.status_code)
print("Response:", response.json())

# ----------------------------
# Predicción batch (todos los objetos)
# ----------------------------
batch_payload = {"batch": payloads}
response = requests.post(f"{BASE_URL}/predict_batch", json=batch_payload)
print("\n=== Batch Prediction ===")
print("Payload:", json.dumps(batch_payload, indent=4))
print("Status Code:", response.status_code)
print("Response:", response.json())

# ----------------------------
# Exportar Excel
# ----------------------------
response = requests.get(f"{BASE_URL}/export_predictions")
print("\n=== Export Predictions ===")
print("Status Code:", response.status_code)
print("Content-Disposition:", response.headers.get("Content-Disposition"))



# Dockerfile
# 1. Imagen base
FROM python:3.11-slim

# 2. Crear directorio de trabajo
WORKDIR /app

# 3. Copiar requirements y código
COPY requirements.txt .
RUN pip install --no-cache-dir -r requirements.txt

COPY . .

# 4. Exponer puerto y comando
EXPOSE 8000
CMD ["uvicorn", "main:app", "--host", "0.0.0.0", "--port", "8000"]



---
## 7) Criterios de evaluación (rúbrica breve)
- **Justificación técnica (25%)**: dataset, métrica, modelo y preprocesamiento argumentados.
- **Calidad del pipeline (20%)**: reproducibilidad y limpieza (Pipeline, persistencia correcta).
- **Contrato y validaciones (25%)**: Pydantic coherente, errores claros, orden de features garantizado.
- **Pruebas (20%)**: variedad de casos, evidencia de resultados y manejo de fallos.
- **Código y documentación (10%)**: legibilidad, estructura y claridad de mensajes.

---
## Bitácora de decisiones (responde aquí)
- Dataset y objetivo:
- Selección de features/target:
- Modelo y preprocesamiento:
- Métrica principal y resultados:
- Decisiones de contrato (payload, validaciones, respuestas):
- Observabilidad y pruebas:
- Lecciones aprendidas:



# Bitácora del Proyecto: Predicción de Churn

---

## 1) Dataset y objetivo

- **Dataset:** Información de clientes de telecomunicaciones, incluyendo características de uso (llamadas, minutos, cargos), planes contratados y llamadas al servicio de atención al cliente.
- **Objetivo:** Predecir si un cliente realizará churn (abandono del servicio) basado en sus patrones de uso y características del plan.
- **Motivación:** Identificar clientes con alto riesgo de churn permite tomar acciones preventivas para retenerlos.

---

## 2) Selección de features/target

- **Target:** `Churn` (0 = no churn, 1 = churn)
- **Features originales:**  
  - `Account_Length`, `Area_Code`, `Intl_Plan`, `Vmail_Plan`, `Vmail_Message`, `CustServ_Calls`, `Day_Mins`, `Eve_Mins`, `Night_Mins`, `Intl_Mins`, `Day_Calls`, `Eve_Calls`, `Night_Calls`, `Intl_Calls`, `Day_Charge`, `Eve_Charge`, `Night_Charge`, `Intl_Charge`
- **Transformaciones y feature engineering:**
  - Codificación binaria de planes: `Intl_Plan`, `Vmail_Plan`
  - Derivadas:  
    - `Total_Calls` = suma de todas las llamadas  
    - `Total_Mins` = suma de minutos totales  
    - `Total_Charge` = suma de cargos totales  
    - `High_Usage` = clientes con `Total_Charge` mayor que el promedio  
    - `Many_CustServ_Calls` = clientes con más de 5 llamadas a servicio
- **Columnas finales:**  

`['Account_Length', 'Area_Code', 'Intl_Plan', 'Vmail_Plan', 'Vmail_Message', 'CustServ_Calls',
'Total_Calls', 'Total_Mins', 'Total_Charge', 'High_Usage', 'Many_CustServ_Calls']`


## 3) Modelo y preprocesamiento

- **Modelo:** Decision Tree Classifier (árbol de decisión)  
- **Parámetros ajustados:**  
- `max_depth = 5` → limita la complejidad para evitar overfitting  
- `min_samples_split = 20` → evita divisiones sobre muestras muy pequeñas  
- `min_samples_leaf = 10` → asegura que cada hoja tenga un mínimo de registros  
- **Preprocesamiento:**  
- Transformación de variables categóricas a numéricas
- Generación de features derivadas
- Eliminación de columnas redundantes
- Eliminación de valores nulos

## 4) Métrica principal y resultados

- **Métricas obtenidas en test:**
- Accuracy: 0.931 → proporción total de predicciones correctas
- Precision: 0.762 → de los clientes predichos como churn, 76% realmente churn
- Recall: 0.764 → de los clientes que hicieron churn, 76% fueron detectados. Detectamos 3 de cada 4 clientes.
- F1-score: 0.762 → balance entre precisión y recall
- **Interpretación:**  
- La métrica principal depende del objetivo de negocio. Para retención, generalmente **recall** es clave: no perder clientes que realmente van a churn.
- La precisión también es importante para no gastar recursos en clientes que no van a churn.
- Nuestro modelo muestra un buen balance entre precisión y recall.



## 5) Decisiones de contrato (payload, validaciones, respuestas)

- **Payload de entrada (`ChurnInput`):**
```json
{
  "Account_Length": 150,
  "Area_Code": 415,
  "Intl_Plan": 1,
  "Vmail_Plan": 0,
  "Vmail_Message": 50,
  "CustServ_Calls": 3,
  "Total_Calls": 500,
  "Total_Mins": 1200,
  "Total_Charge": 250,
  "High_Usage": 1,
  "Many_CustServ_Calls": 0
}```


Validaciones:

Rangos para cada campo
Tipos numéricos
Campos obligatorios

```json response
{
  "predicted_churn": 1,
  "probability_churn": 0.78,
  "warnings": ["CustServ_Calls fuera de rango típico"]
}
```


## 6) Observabilidad y pruebas

- Middleware de latencia: añade header X-Process-Time-ms a cada respuesta
- Logger estructurado: registro de cada request con ruta, método, status y tiempo de procesamiento
- Pruebas realizadas:
- Casos válidos: 3 (diferentes perfiles de clientes)
- Casos inválidos: 3 (campo faltante, tipo incorrecto, campo extra)
- Verificación de probabilidades y predicciones
- Endpoint adicional: /export_predictions
- Genera un Excel con los datos originales, las features derivadas y las predicciones + probabilidades



## 7) Lecciones aprendidas

- La selección de features derivadas (Total_Calls, Total_Charge, High_Usage) mejoró el desempeño del modelo sin agregar complejidad innecesaria.
- Ajustar min_samples_split y min_samples_leaf permite balancear overfitting y precisión.
- Es crítico definir claramente el payload y validaciones para APIs de ML, evitando errores por desorden de columnas o tipos de datos.
- Medir tanto precision como recall ayuda a decidir umbrales de acción para negocio.
- Registrar métricas de latencia y warnings incrementa la confiabilidad y explicabilidad de la API.
- La documentación clara y el pipeline reproducible facilitan la entrega y el entendimiento por terceros.






